In [1]:
import numpy as np

In [2]:
def power_perp_price(spot, time, vol, drift, power):
    price = spot ** power * np.exp((power - 1) * (drift + power / 2 * vol ** 2) * time)
    return price


def everlasting_power_perp_price(spot, funding_period, vol, drift, power):
    price = spot ** power * (1 / (2* np.exp(-funding_period/2 * (power - 1) * (2 * drift + power * vol **2)) - 1))
    return price


def get_sample(spot, time, vol, drift):
    """
    Computes new price randomly based on a starting price, a drift, a volatility, and a duration
    See: https://en.wikipedia.org/wiki/Geometric_Brownian_motion for a derivation
    """
    
    # Compute a random standard normal 
    rand = np.random.normal(loc=0.0, scale=1.0)

    # Calculate new price based on a geometric brownian motion model
    newPrice = spot * np.exp(((drift - (0.5 * (vol ** 2))) * time) + (vol * np.sqrt(time) * rand))
    
    return newPrice


def get_samples(num_samples, spot, time, vol, drift):
    """
    Get a set of samples from this GBM where spot is the starting price, time is the number of periods,
    vol is the percentage volatility, drift is the percentage drift, and num_samples is the number of samples. 
    """
    samples = np.array([get_sample(spot, time, vol, drift) for _ in range(num_samples)])
    
    return samples


def cnd(d):
    """ CDF of the normal distribution """
    A1 = 0.31938153
    A2 = -0.356563782
    A3 = 1.781477937
    A4 = -1.821255978
    A5 = 1.330274429
    RSQRT2PI = 0.39894228040143267793994605993438
    K = 1.0 / (1.0 + 0.2316419 * np.abs(d))
    ret_val = (RSQRT2PI * np.exp(-0.5 * d * d) *
               (K * (A1 + K * (A2 + K * (A3 + K * (A4 + K * A5))))))
    return np.where(d > 0, 1.0 - ret_val, ret_val)


def black_scholes(stockPrice, optionStrike, optionYears, riskFreeRate, volatility, callOrPut):
    
    # spot price of an asset
    S = stockPrice
    
    # strike price
    K = optionStrike
    
    # time to maturity in years
    T = optionYears
    
    # risk-free interest rate
    R = riskFreeRate
    
    # volatility of the asset
    V = volatility
    
    # To simplify the calculation, d1 and d2 are typically calculated seperatesly and the the cumulative distribution functions of d1 and d2 are taken
    d1 = (np.log(S / K) + (R + 0.5 * V * V) * T) / (V * np.sqrt(T))
    d2 = d1 - V * np.sqrt(T)
    
    # Finally, the result for the call is calculated
    if callOrPut == 'call':
        optionPrice = S * cnd(d1) - K * np.exp(-R*T) * cnd(d2)
    elif callOrPut == 'put':
        optionPrice = -S * cnd(-d1) + K * np.exp(-R*T) * cnd(-d2)

    return optionPrice

In [3]:
# Demonstrate sampling gives the same result as black scholes
def test_black_scholes():
    num_samples = 10000
    spot = 2
    time = 0.5
    vol = 0.8
    drift = 0.2
    samples = get_samples(num_samples,spot,time,vol,drift)

    strike = spot * 1.1

    discounted_payoff = np.mean(np.maximum(samples-strike,0) * np.exp(time*drift*-1))

    calced = black_scholes(spot, strike, time, drift, vol)

    assert np.abs(discounted_payoff - calced)/calced < 0.05, (discounted_payoff, calced)

# Demonstrate our power perp pricing is the same as sampling
def test_power_perp():
    num_samples = 100000
    spot = 2
    time = 0.5
    vol = 1.2
    drift = 0.2
    power = 3
    samples = get_samples(num_samples,spot,time,vol,drift)

    discounted_payoff = np.mean(samples**power * np.exp(time*drift*-1))

    calced = power_perp_price(spot, time, vol, drift, power)

    assert np.abs(discounted_payoff - calced)/calced < 0.05, (discounted_payoff, calced)

def test_everlasting_power_perp():
    spot = 2
    vol = 1.2
    drift = 0.2
    power = 3
    funding_period = 1/7

    num_iters = 1000
    est = 0
    for i in range(1,num_iters,1):
        est += power_perp_price(spot, i * funding_period, vol, drift, power) / 2**i

    calced = everlasting_power_perp_price(spot, funding_period, vol, drift, power)

    assert np.abs(est - calced)/est < 0.05, (est, calced)

In [4]:
test_everlasting_power_perp()

for period in range(1,100):
    print((period,everlasting_power_perp_price(2,1/period,1.2,0,3)))

(1, -8.218613196795545)
(2, -10.398391521677164)
(3, -15.20494895227915)
(4, -24.93696024430005)
(5, -50.937773824258954)
(6, -301.9382007920685)
(7, 101.30783479432498)
(8, 48.33938947642748)
(9, 33.674741385800445)
(10, 26.80796668992566)
(11, 22.829373019670456)
(12, 20.235098858336126)
(13, 18.410561755790233)
(14, 17.057897625472073)
(15, 16.015272661185186)
(16, 15.187211176296588)
(17, 14.513759157788748)
(18, 13.955377488831141)
(19, 13.484938474467635)
(20, 13.083212751320724)
(21, 12.736191345537407)
(22, 12.433427718187096)
(23, 12.166973706882452)
(24, 11.930675210191124)
(25, 11.719693325733614)
(26, 11.530171032583281)
(27, 11.358996311543082)
(28, 11.203630657739193)
(29, 11.06198285743313)
(30, 10.93231468180441)
(31, 10.813169465651331)
(32, 10.703317345874924)
(33, 10.601712797039434)
(34, 10.507461359589012)
(35, 10.419793320589182)
(36, 10.338042709602435)
(37, 10.261630398569674)
(38, 10.190050399949182)
(39, 10.122858678782597)
(40, 10.059663956698744)
(41, 10.000